In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load the dataset
df = pd.read_csv('BostonHousing.csv')
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [15]:
#Check for duplicates
n_duplicates = df.duplicated().sum()
print(f"Found {n_duplicates} duplicate rows.")

Found 0 duplicate rows.


In [16]:
# Check for missing values in all columns
print("\nMissing values per column:")
print(df.isnull().sum())



Missing values per column:
crim       0
zn         0
indus      0
chas       0
nox        0
rm         0
age        0
dis        0
rad        0
tax        0
ptratio    0
b          0
lstat      0
medv       0
dtype: int64


In [ ]:
# Check for missing values in rows
null_rows_df = df[df.isnull().any(axis=1)]
null_rows_df

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv


In [18]:
from scipy import stats

numeric_cols = [
    'crim', 'zn', 'indus', 'nox', 'rm', 'age', 'dis', 'tax', 'ptratio', 'b', 'lstat', 'medv'
]

outlier_indices = dict()

for col in numeric_cols:
    skewness = df[col].skew()
    print(f"\nColumn: {col:<10} skewness = {skewness:.2f}")
    
    if abs(skewness) < 0.5:
        # approximately normal → Z-score method
        zscores = np.abs(stats.zscore(df[col]))
        mask = zscores > 3
        method = 'Z-score > 3'
    else:
        # skewed → IQR method
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
        mask = (df[col] < lower) | (df[col] > upper)
        method = 'IQR 1.5× range'
    
    idx = df[mask].index
    outlier_indices[col] = idx
    print(f"  Detected {len(idx)} outliers by {method}.")



Column: crim       skewness = 5.22
  Detected 66 outliers by IQR 1.5× range.

Column: zn         skewness = 2.23
  Detected 68 outliers by IQR 1.5× range.

Column: indus      skewness = 0.30
  Detected 0 outliers by Z-score > 3.

Column: nox        skewness = 0.73
  Detected 0 outliers by IQR 1.5× range.

Column: rm         skewness = 0.40
  Detected 8 outliers by Z-score > 3.

Column: age        skewness = -0.60
  Detected 0 outliers by IQR 1.5× range.

Column: dis        skewness = 1.01
  Detected 5 outliers by IQR 1.5× range.

Column: tax        skewness = 0.67
  Detected 0 outliers by IQR 1.5× range.

Column: ptratio    skewness = -0.80
  Detected 15 outliers by IQR 1.5× range.

Column: b          skewness = -2.89
  Detected 77 outliers by IQR 1.5× range.

Column: lstat      skewness = 0.91
  Detected 7 outliers by IQR 1.5× range.

Column: medv       skewness = 1.11
  Detected 40 outliers by IQR 1.5× range.
